In [31]:
import pandas as pd
from rdkit import Chem
from tqdm import tqdm

import numpy as np

from datasets import Hergophores

In [32]:
DATA = "data_filtered.tsv"

df = pd.read_csv(DATA, sep="\t", dtype=str)


In [33]:
unique_hergophores, _ = Hergophores.get()

matches = np.zeros((len(df), len(unique_hergophores)))
atom_match_counts = []

for i, smile in tqdm(enumerate(df["smiles"].tolist())):
    mol = Chem.MolFromSmiles(smile)
    num_atoms = mol.GetNumAtoms()

    atom_match_count = np.zeros(num_atoms)

    for j, hergophore in enumerate(unique_hergophores):
        mol_hergophore = Chem.MolFromSmiles(hergophore)

        match = mol.GetSubstructMatch(mol_hergophore)

        if match:
            matches[i, j] = 1

            atom_match = np.array([1 if i in match else 0 for i in range(num_atoms)])
            atom_match_count += atom_match

    atom_match_counts.append(atom_match_count)

N_matches = np.sum(matches, axis=1)
df["N_matches"] = N_matches
df["atom_match_count"] = atom_match_counts

for j, hergophore in enumerate(unique_hergophores):
    df[hergophore] = matches[:, j]


7858it [00:06, 1198.83it/s]


In [34]:
df.to_csv("data_filtered_matches.tsv", index=False, sep="\t")

In [35]:
more_than_one = len(np.where(N_matches > 1)[0])

print(more_than_one)






6426
